# Run Experiments on Fjord

## Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
from collections import defaultdict
from anytree.importer import JsonImporter
from utils.custom_dataset import CustomDataset
from ultralytics import RTDETR
import contextlib
import io

# extract test data
df = pd.read_parquet('datasets/yolov8-segmented-objects-dataset.parquet')

# Assuming df is your DataFrame with all data
train_val_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

# Local version of get labels
def get_hierarchical_labels(species_index, species_names, genus_names, class_names, binary_names, root):
    if species_index == -1:
        return -1, -1, -1  # Handle cases where species_index is invalid

    species_name = species_names[species_index]
    node = next((n for n in root.descendants if n.name == species_name), None)

    if node is None:
        return -1, -1, -1  # Species not found in the tree

    genus_index, class_index, binary_index = -1, -1, -1
    current_node = node
    while current_node.parent is not None:
        current_node = current_node.parent
        if current_node.rank == 'genus':
            genus_index = genus_names.index(current_node.name) if current_node.name in genus_names else -1
        elif current_node.rank == 'class':
            class_index = class_names.index(current_node.name) if current_node.name in class_names else -1
        elif current_node.rank == 'binary':
            binary_index = binary_names.index(current_node.name) if current_node.name in binary_names else -1

    return genus_index, class_index, binary_index

In [2]:
model_path = "datasets/hierarchical-model-weights/weights/best_model_alpha_0.50.pth"
# Load the state dictionary
state_dict = torch.load(model_path)

# Print the keys of the state dictionary
print(state_dict.keys())

odict_keys(['conv1.0.weight', 'conv1.0.bias', 'conv1.1.weight', 'conv1.1.bias', 'conv1.1.running_mean', 'conv1.1.running_var', 'conv1.1.num_batches_tracked', 'conv1.3.weight', 'conv1.3.bias', 'conv1.4.weight', 'conv1.4.bias', 'conv1.4.running_mean', 'conv1.4.running_var', 'conv1.4.num_batches_tracked', 'conv1.7.fc.0.weight', 'conv1.7.fc.2.weight', 'conv1.8.conv1.weight', 'conv2.0.weight', 'conv2.0.bias', 'conv2.1.weight', 'conv2.1.bias', 'conv2.1.running_mean', 'conv2.1.running_var', 'conv2.1.num_batches_tracked', 'conv2.3.weight', 'conv2.3.bias', 'conv2.4.weight', 'conv2.4.bias', 'conv2.4.running_mean', 'conv2.4.running_var', 'conv2.4.num_batches_tracked', 'conv2.7.weight', 'conv2.7.bias', 'conv2.8.weight', 'conv2.8.bias', 'conv2.8.running_mean', 'conv2.8.running_var', 'conv2.8.num_batches_tracked', 'conv2.10.fc.0.weight', 'conv2.10.fc.2.weight', 'conv2.11.conv1.weight', 'conv3.0.weight', 'conv3.0.bias', 'conv3.1.weight', 'conv3.1.bias', 'conv3.1.running_mean', 'conv3.1.running_var', 

## Load Model and run experiments

In [3]:
def run_experiment(data_path, classes_file, model_path, ablation, root, reclassify=True):
    df = pd.read_parquet(data_path)
    _, test_df = train_test_split(df, test_size=0.3, random_state=42)

    object_names = [line.strip() for line in open(classes_file, 'r')]
    subcategory_names, category_names, binary_names = [], [], []
    for node in root.descendants:
        if node.rank == 'genus':
            subcategory_names.append(node.name)
        elif node.rank == 'class':
            category_names.append(node.name)
        elif node.rank == 'binary':
            binary_names.append(node.name)

    rank_counts = defaultdict(int)
    for node in root.descendants:
        rank_counts[node.rank] += 1
    num_classes_hierarchy = [rank_counts['binary'], rank_counts['class'], rank_counts['genus'], rank_counts['species']]

    # Select the appropriate model based on the ablation
    if ablation in ['yolov8', 'baseline', 'yolov9', 'RT-DETR']:
        from models.hierarchical_cnn import HierarchicalCNN
        model = HierarchicalCNN(num_classes_hierarchy, num_additional_features=2)
    elif ablation == 'remove_features':
        from models.ablations.remove_features.hierarchical_cnn import HierarchicalCNN
        model = HierarchicalCNN(num_classes_hierarchy)
    elif ablation == 'attention_removed':
        from models.ablations.attention_removed.hierarchical_cnn import HierarchicalCNN
        model = HierarchicalCNN(num_classes_hierarchy, num_additional_features=2)
    elif ablation == 'decreased_branch_complexity':
        from models.ablations.decreased_branch_complexity.hierarchical_cnn import HierarchicalCNN
        model = HierarchicalCNN(num_classes_hierarchy, num_additional_features=2)
    elif ablation == 'increased_features_complexity':
        from models.ablations.increased_features_complexity.hierarchical_cnn import HierarchicalCNN
        model = HierarchicalCNN(num_classes_hierarchy, num_additional_features=2)
    else:
        raise ValueError(f"Unsupported ablation study: {ablation}")

    model.load_state_dict(torch.load(model_path))
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()

    test_dataset = CustomDataset(test_df, object_names, subcategory_names, category_names, binary_names, root)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    true_labels = {level: [] for level in ['binary', 'class', 'genus', 'species']}
    predictions = {level: [] for level in ['binary', 'class', 'genus', 'species']}

    with torch.no_grad():
        for images, conf, pred_species, species_index, genus_index, class_index, binary_index in test_loader:
            images, conf, pred_species = [x.to(device) for x in [images, conf, pred_species]]

            if not reclassify:
                for pred in pred_species:
                    genus_idx, class_idx, binary_idx = get_hierarchical_labels(pred.item(), object_names, subcategory_names, category_names, binary_names, root)
                    predictions['species'].append(pred.item())
                    predictions['genus'].append(genus_idx)
                    predictions['class'].append(class_idx)
                    predictions['binary'].append(binary_idx)
            else:
                outputs = model(images, conf, pred_species)
                for i, output in enumerate(outputs):
                    _, predicted = torch.max(output, 1)
                    level = ['binary', 'class', 'genus', 'species'][i]
                    predictions[level].extend(predicted.cpu().numpy())

            true_labels['binary'].extend(binary_index.cpu().numpy())
            true_labels['class'].extend(class_index.cpu().numpy())
            true_labels['genus'].extend(genus_index.cpu().numpy())
            true_labels['species'].extend(species_index.cpu().numpy())

    # Remove cases where no predictions were made
    for level in ['binary', 'class', 'genus', 'species']:
        true_labels[level] = [label for label, pred in zip(true_labels[level], predictions[level]) if pred != -1]
        predictions[level] = [pred for pred in predictions[level] if pred != -1]

    f1_scores = {level: f1_score(true_labels[level], predictions[level], average='macro') for level in ['binary', 'class', 'genus', 'species']}
    return f1_scores

# Populate Taxonomy
importer = JsonImporter()
with open('datasets/ontology.json', 'r') as f:
    root = importer.read(f)

# Paths and setup
classes_file = 'datasets/EMVSD/EMVSD/classes.txt'
ablations = ['yolov8', 'yolov9', 'RT-DETR', 'attention_removed', 'decreased_branch_complexity', 'increased_features_complexity', 'remove_features']
alpha_values = [0, 0.2, 0.5, 0.8, 1]

# Run experiments and collect results
results = []
for ablation in ablations:
    for alpha in alpha_values:
        if ablation in ['yolov8', 'baseline']:
            data_path = 'datasets/yolov8-segmented-objects-dataset.parquet'
            model_path = f'datasets/hierarchical-model-weights/weights/best_model_alpha_{alpha:.2f}.pth'
        elif ablation == 'RT-DETR':
            data_path = 'datasets/rtdetr-segmented-objects-dataset.parquet'
            model_path = f'datasets/hierarchical-model-weights/weights/best_model_alpha_rtdetr_{alpha:.2f}.pth'
        elif ablation == 'yolov9':
            data_path = 'datasets/yolov9-segmented-objects-dataset.parquet'
            model_path = f'datasets/hierarchical-model-weights/weights/best_model_alpha_yolov9_{alpha:.2f}.pth'
        else:
            data_path = 'datasets/yolov8-segmented-objects-dataset.parquet'
            model_path = f'datasets/hierarchical-model-weights/ablations/{ablation}/weights/best_model_alpha_{alpha:.2f}.pth'
        
        # Run experiment without reclassification
        f1_scores_no_reclassification = run_experiment(data_path, classes_file, model_path, ablation, root, reclassify=False)
        results.append({'Ablation': ablation + '-no-reclassification', 'Alpha': alpha, **f1_scores_no_reclassification})

        # Run experiment with reclassification for hierarchical CNN models
        if ablation in ['yolov8', 'RT-DETR', 'yolov9']:
            f1_scores_reclassified = run_experiment(data_path, classes_file, model_path, ablation, root, reclassify=True)
            results.append({'Ablation': ablation + '-Reclassified', 'Alpha': alpha, **f1_scores_reclassified})

# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df.round(4)

,Ablation,Alpha,binary,class,genus,species
0,yolov8-no-reclassification,0.0,0.6612,0.8792,0.9131,0.9185
1,yolov8-Reclassified,0.0,0.6460,0.8685,0.9021,0.9073
2,yolov8-no-reclassification,0.2,0.6612,0.8792,0.9131,0.9185
3,yolov8-Reclassified,0.2,0.6511,0.8687,0.9034,0.9101
4,yolov8-no-reclassification,0.5,0.6612,0.8792,0.9131,0.9185
5,yolov8-Reclassified,0.5,0.6491,0.8668,0.9008,0.9076
6,yolov8-no-reclassification,0.8,0.6612,0.8792,0.9131,0.9185
7,yolov8-Reclassified,0.8,0.6489,0.8717,0.9055,0.9119
8,yolov8-no-reclassification,1.0,0.6612,0.8792,0.9131,0.9185
9,yolov8-Reclassified,1.0,0.6462,0.8694,0.9038,0.9070
